In [ ]:
from pymbe.api import SysML2ClientWidget

In [ ]:
client = SysML2ClientWidget()
client

In [ ]:
client.project_selector.value = "03e3cdfe-8d90-4fc3-a414-9a188203d349"
client._download_elements()

In [ ]:
[
    element["qualifiedName"]
    for element in client.selected_elements_by_type
]

# Scratch Pad

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation